In [ ]:
import eodslib
from datetime import datetime
from pathlib import Path

In [ ]:
# set the connection dictionary
conn = {
    'domain': 'https://earthobs.defra.gov.uk',
    'username': '<insert-user-name>',
    'access_token': '<insert-user-token>',
    }

In [ ]:
# set the user defined output directory for WPS and logging results
output_dir = Path('/some/directory/')

In [ ]:
"""
You can also use the config.py to import these settings and apply them across any notebook
See below (commented out) example on how to import settings
"""
# import config
# conn = config.conn
# output_dir = config.output_dir

In [ ]:
# example of keyword argument (kwarg) expansion, the input needs to be a python dictionary

# query a specific S2 granule but limit the number to 2
eods_params = {
    'output_dir':output_dir,
    'title':'T30UXG_ORB080',
    'limit':2,
    }

list_of_layers, df = eodslib.query_catalog(conn, **eods_params)

In [ ]:
# create an empty list of results
list_of_results = list()

# iterative through the returned EODS query list of layers
for lyr in list_of_layers:

    # for each layer, create a wps config dictionary, inserting the layer name
    config_wpsprocess = {'template_xml':'gsdownload_template.xml',
        'xml_config':{
            'template_layer_name':lyr,
            'template_outputformat':'image/tiff',
            'template_mimetype':'application/zip'
                },
        'dl_bool':True
    }

    # for each layer, call the run wps function which:
        # submits the WPS job
        # polls the wps server until the job is READY
        # downloads the result (and retries up to 3 times if there's a break in the download connection)
    execution_dict = eodslib.run_wps(conn, config_wpsprocess, output_dir=output_dir)

    # append the execution dictionary to the results list
    list_of_results.append(execution_dict)

# log the output to a log csv file
eodslib.output_log(list_of_results)

In [ ]:
# do something else (process locally) using the list_of_results which contains the local path of any wps output

In [ ]:
print('### Script Finished')